In [21]:
import pandas as pd
from collections import defaultdict

## Chargement des données

In [22]:
routes = pd.read_csv('IDFM-gtfs/routes.txt')
trips = pd.read_csv('IDFM-gtfs/trips.txt')
stop_times = pd.read_csv('IDFM-gtfs/stop_times.txt')
stops = pd.read_csv('IDFM-gtfs/stops.txt')

/var/folders/0y/kz93z2gx1fn6s9_d1wp7pq300000gn/T/ipykernel_69486/1799971113.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv('IDFM-gtfs/trips.txt')


In [23]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order
0,IDFM:C01371,IDFM:Operator_100,1,1,NaN,1,NaN,FFBE00,000000,NaN
1,IDFM:C01286,IDFM:Operator_100,322,322,NaN,3,NaN,82C8E6,000000,NaN
2,IDFM:C01153,IDFM:Operator_100,124,124,NaN,3,NaN,FF82B4,000000,NaN
3,IDFM:C00173,IDFM:1081,18,18,NaN,3,NaN,DB006B,FFFFFF,NaN
4,IDFM:C00391,IDFM:1058,4334,4334,NaN,3,NaN,E2001A,FFFFFF,NaN


## Filtrage des données

### Lignes et parcours

In [24]:
# Sélection des lignes et parcours de metro uniquement
metro_routes = routes[routes['route_type'] == 1]['route_id']
metro_trips = trips[trips['route_id'].isin(metro_routes)]

In [25]:
metro_trips.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
49891,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49892,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_3,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49894,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49900,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_5,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
49901,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_7,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0


In [26]:
# On sélectionne les 16 premiers parcours de chaque ligne afin d'inclure tout les chemins possibles
selected_trips = metro_trips.groupby(['route_id', 'direction_id']).head(16).reset_index()

selected_trips.head()

,index,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,49891,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
1,49892,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_3,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
2,49894,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_95549...,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
3,49900,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_5,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0
4,49901,IDFM:C01371,IDFM:1,IDFM:RATP:107769-C01371-COU_RATP_4864461_955496_7,La Défense (Grande Arche),NaN,1,NaN,NaN,2,0


### Arrêts

In [27]:
stop_times = stop_times[stop_times['trip_id'].isin(selected_trips['trip_id'])]
stop_times = stop_times.sort_values(['trip_id', 'stop_sequence'])
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint
1432168,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:30:00,07:30:00,IDFM:22175,0,0,1,NaN,NaN,1
1432169,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:31:00,07:31:00,IDFM:22173,1,0,0,NaN,NaN,1
1432170,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:32:00,07:32:00,IDFM:22160,2,0,0,NaN,NaN,1
1432171,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:34:00,07:34:00,IDFM:22158,3,0,0,NaN,NaN,1
1432172,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:35:00,07:35:00,IDFM:22155,4,0,0,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
1432695,IDFM:RATP:130780-C01376-COU_RATP_5096756_29914...,21:25:00,21:25:00,IDFM:22165,23,0,0,NaN,NaN,1
1432696,IDFM:RATP:130780-C01376-COU_RATP_5096756_29914...,21:26:00,21:26:00,IDFM:22164,24,0,0,NaN,NaN,1
1432697,IDFM:RATP:130780-C01376-COU_RATP_5096756_29914...,21:27:00,21:27:00,IDFM:463114,25,0,0,NaN,NaN,1
1432698,IDFM:RATP:130780-C01376-COU_RATP_5096756_29914...,21:28:00,21:28:00,IDFM:463102,26,0,0,NaN,NaN,1


In [28]:
def to_seconds(t):
    h, m, s = map(int, t.split(":"))
    return h * 3600 + m * 60 + s

stop_times['arr_sec'] = stop_times['arrival_time'].apply(to_seconds)
stop_times['dep_sec'] = stop_times['departure_time'].apply(to_seconds)

In [29]:
physical_stops = stops[stops["location_type"] == 0]

In [30]:
stop_times = stop_times.merge(physical_stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']], on='stop_id', how='left')
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint,arr_sec,dep_sec,stop_name,stop_lat,stop_lon
0,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:30:00,07:30:00,IDFM:22175,0,0,1,NaN,NaN,1,27000,27000,Corvisart,48.829860,2.350611
1,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:31:00,07:31:00,IDFM:22173,1,0,0,NaN,NaN,1,27060,27060,Glacière,48.831170,2.343497
2,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:32:00,07:32:00,IDFM:22160,2,0,0,NaN,NaN,1,27120,27120,Saint-Jacques,48.832916,2.337154
3,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:34:00,07:34:00,IDFM:22158,3,0,0,NaN,NaN,1,27240,27240,Denfert-Rochereau,48.834313,2.332872
4,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:35:00,07:35:00,IDFM:22155,4,0,0,NaN,NaN,1,27300,27300,Raspail,48.839085,2.330550


In [31]:
stop_times['next_stop_name'] = stop_times.groupby('trip_id')['stop_name'].shift(-1)
stop_times['next_arr_sec'] = stop_times.groupby('trip_id')['arr_sec'].shift(-1)

In [32]:
stop_times[stop_times['stop_name'] == 'Châtelet']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint,arr_sec,dep_sec,stop_name,stop_lat,stop_lon,next_stop_name,next_arr_sec
85,IDFM:RATP:107499-C01377-COU_RATP_5083921_24678...,13:06:00,13:06:00,IDFM:463060,19,0,0,NaN,NaN,1,47160,47160,Châtelet,48.856868,2.348102,Pont Marie (Cité des Arts),47280.0
118,IDFM:RATP:107499-C01377-COU_RATP_5083921_24678...,13:13:00,13:13:00,IDFM:463060,19,0,0,NaN,NaN,1,47580,47580,Châtelet,48.856868,2.348102,Pont Marie (Cité des Arts),47700.0
151,IDFM:RATP:107499-C01377-COU_RATP_5083921_24678...,07:03:00,07:03:00,IDFM:463060,19,0,0,NaN,NaN,1,25380,25380,Châtelet,48.856868,2.348102,Pont Marie (Cité des Arts),25440.0
184,IDFM:RATP:107499-C01377-COU_RATP_5083921_24678...,13:20:00,13:20:00,IDFM:463060,19,0,0,NaN,NaN,1,48000,48000,Châtelet,48.856868,2.348102,Pont Marie (Cité des Arts),48120.0
217,IDFM:RATP:107499-C01377-COU_RATP_5083921_24678...,13:27:00,13:27:00,IDFM:463060,19,0,0,NaN,NaN,1,48420,48420,Châtelet,48.856868,2.348102,Pont Marie (Cité des Arts),48540.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9926,IDFM:RATP:130697-C01374-COU_RATP_5097971_31557...,24:36:00,24:36:00,IDFM:463158,16,0,0,NaN,NaN,1,88560,88560,Châtelet,48.859716,2.346760,Les Halles,88620.0
9955,IDFM:RATP:130697-C01374-COU_RATP_5097971_31557...,24:41:00,24:41:00,IDFM:463158,16,0,0,NaN,NaN,1,88860,88860,Châtelet,48.859716,2.346760,Les Halles,88920.0
9984,IDFM:RATP:130697-C01374-COU_RATP_5097971_31557...,24:46:00,24:46:00,IDFM:463158,16,0,0,NaN,NaN,1,89160,89160,Châtelet,48.859716,2.346760,Les Halles,89220.0
10013,IDFM:RATP:130697-C01374-COU_RATP_5097971_31557...,24:51:00,24:51:00,IDFM:463158,16,0,0,NaN,NaN,1,89460,89460,Châtelet,48.859716,2.346760,Les Halles,89520.0


## Construction du graphe

In [33]:
edges = stop_times.dropna(subset=['next_stop_name']).copy()
edges['duration'] = edges['next_arr_sec'] - edges['dep_sec']

In [34]:
edges.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint,arr_sec,dep_sec,stop_name,stop_lat,stop_lon,next_stop_name,next_arr_sec,duration
0,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:30:00,07:30:00,IDFM:22175,0,0,1,NaN,NaN,1,27000,27000,Corvisart,48.829860,2.350611,Glacière,27060.0,60.0
1,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:31:00,07:31:00,IDFM:22173,1,0,0,NaN,NaN,1,27060,27060,Glacière,48.831170,2.343497,Saint-Jacques,27120.0,60.0
2,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:32:00,07:32:00,IDFM:22160,2,0,0,NaN,NaN,1,27120,27120,Saint-Jacques,48.832916,2.337154,Denfert-Rochereau,27240.0,120.0
3,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:34:00,07:34:00,IDFM:22158,3,0,0,NaN,NaN,1,27240,27240,Denfert-Rochereau,48.834313,2.332872,Raspail,27300.0,60.0
4,IDFM:RATP:107494-C01376-COU_RATP_5074966_11992...,07:35:00,07:35:00,IDFM:22155,4,0,0,NaN,NaN,1,27300,27300,Raspail,48.839085,2.330550,Edgar Quinet,27360.0,60.0


In [35]:
edges = edges[['stop_name', 'next_stop_name', 'duration']]
edges.columns = ['u', 'v', 'w']
edges.head()

,u,v,w
0,Corvisart,Glacière,60.0
1,Glacière,Saint-Jacques,60.0
2,Saint-Jacques,Denfert-Rochereau,120.0
3,Denfert-Rochereau,Raspail,60.0
4,Raspail,Edgar Quinet,60.0


In [36]:
graph = defaultdict(dict)
for row in edges.itertuples(index=False):
    u, v, w = row.u, row.v, row.w
    if v not in graph[u] or graph[u][v] > w:
        graph[u][v] = w
        graph[v][u] = w
        

### Sauvegarde du graphe

In [37]:
with open("graph.py", "w") as f:
    f.write("from collections import defaultdict\n")
    f.write("graph = defaultdict(dict)\n")
    for u, neighbors in graph.items():
        for v, w in neighbors.items():
            f.write(f"graph[{repr(u)}][{repr(v)}] = {w}\n")

### Tests du graphe

In [38]:
def is_connected(graph):
    if not graph:
        return True  # An empty graph is trivially connected

    visited = set()
    start = next(iter(graph))  # Start from any node

    def dfs(node):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor)

    dfs(start)
    
    return (len(visited) == len(graph), visited)


In [39]:
def test_connexite(graph):
    est_connnexe, visited = is_connected(graph)
    if not est_connnexe:
        for g in graph:
            if g not in visited:
                print(g)
    else:
        print('Le graphe est connexe.')

test_connexite(graph)

Le graphe est connexe.


In [40]:
len(graph)

308